In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
ev_stations = pd.read_csv("data/alt_fuel_stations.csv")

/var/folders/57/ldpq7gs518dcdcrx7jr7m1440000gn/T/ipykernel_69563/3738557161.py:1: DtypeWarning: Columns (6,20,31,46,69) have mixed types. Specify dtype option on import or set low_memory=False.
  ev_stations = pd.read_csv("data/alt_fuel_stations.csv")


In [3]:
clean_stations = ev_stations.drop(columns={"Plus4", "Expected Date", "BD Blends", "NG Fill Type Code", "NG PSI", "Federal Agency ID", "Federal Agency Name", "Hydrogen Status Link", "NG Vehicle Class", "LPG Primary", "E85 Blender Pump", "Intersection Directions (French)", "Access Days Time (French)", "BD Blends", "Hydrogen Is Retail", "Federal Agency Code", "CNG Dispenser Num", "CNG On-Site Renewable Source", "CNG Total Compression Capacity", "CNG Storage Capacity", "LNG On-Site Renewable Source", "E85 Other Ethanol Blends", "EV Pricing (French)", "LPG Nozzle Types", "Hydrogen Pressures", "Hydrogen Standards", "CNG Fill Type Code", "CNG PSI", "CNG Vehicle Class", "LNG Vehicle Class"}) 

In [4]:
stations_less_columns = clean_stations.drop(columns={"Intersection Directions", "RD Blends", "RD Blended with Biodiesel", "RD Maximum Biodiesel Level", "NPS Unit Name", "CNG Station Sells Renewable Natural Gas", "LNG Station Sells Renewable Natural Gas", "Maximum Vehicle Class"})
ev_clean_stations = stations_less_columns.drop(columns={"BD Blends (French)", "Access Code", "Access Detail Code", "EV On-Site Renewable Source", "Restricted Access", "RD Blends (French)", "Groups With Access Code (French)"})
ev_clean_stations

,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Station Phone,Status Code,Groups With Access Code,Access Days Time,...,Date Last Confirmed,ID,Updated At,Owner Type Code,Open Date,EV Connector Types,Country,Facility Type,EV Pricing,EV Workplace Charging
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,5:30am-9pm; pay lot,...,2023-01-10,1523,2023-02-14 15:54:11 UTC,P,1995-08-30,J1772,US,PARKING_GARAGE,Free; parking fee,False
1,ELEC,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,626-575-6800,E,Public,24 hours daily,...,2022-09-14,1583,2023-02-14 15:54:11 UTC,SG,1996-10-15,J1772,US,STATE_GOV,Free,False
2,ELEC,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,NaN,E,Public,24 hours daily,...,2023-01-10,6355,2023-02-14 15:54:11 UTC,P,1997-07-30,J1772,US,HOSPITAL,Free; parking fee,False
3,ELEC,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,800-256-6219,E,Public - Call ahead,Dealership business hours; customer use only,...,2023-08-10,6405,2024-01-31 22:07:01 UTC,P,2012-12-11,J1772,US,CAR_DEALER,NaN,False
4,ELEC,Galleria at Tyler,1299 Galleria at Tyler,Riverside,CA,92503,951-351-3110,E,Public,6am-12am daily,...,2023-09-14,6425,2024-01-31 22:07:01 UTC,P,1997-08-30,J1772,US,SHOPPING_MALL,Free,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62322,ELEC,Riverview - 17th Ave SW,7021 17th Ave SW,Seattle,WA,98106,855-900-7584,E,Public,24 hours daily,...,2024-04-02,331532,2024-04-02 02:18:25 UTC,NaN,2024-04-02,J1772,US,NaN,NaN,False
62323,ELEC,Kapolei Commons provided by Hawaiian Electric ...,4470 Kapolei Parkway,Kapolei,HI,96707,855-885-9571,E,Public,NaN,...,2024-04-02,331533,2024-04-02 02:25:15 UTC,NaN,2024-04-02,CHADEMO J1772COMBO,US,NaN,NaN,False
62324,ELEC,Orange County Health Department,12050 E Colonial Dr,Orlando,FL,32826,866-816-7584,E,Public,NaN,...,2024-04-02,331535,2024-04-02 02:29:34 UTC,NaN,2024-04-02,CHADEMO J1772COMBO,US,NaN,NaN,False
62325,ELEC,Cedar Fair Entertainment Company 14523 Carowin...,14523 Carowinds Boulevard,Charlotte,SC,28273,888-264-2208,E,Public,Mon 5:00am - 10:59pm; Tue 5:00am - 10:59pm; We...,...,2024-04-02,331537,2024-04-02 02:33:28 UTC,NaN,2024-04-02,J1772,US,NaN,NaN,False


In [5]:
ev_filled = ev_clean_stations.fillna(0)
full_and_clean = ev_filled.rename(columns={
    "Station Phone": "Phone Number",
    "Groups With Access Code": "Who Can Access",
    "Access Days Time": "Hours of Operation",
    "Cards Accepted": "Cards Accepted?",
    "EV Level1 EVSE Num": "# of EV Level 1 Chargers",
    "EV Level2 EVSE Num": "# of EV Level 2 Chargers",
    "EV DC Fast Count": "# of DC EV Chargers",
    "EV Network Web": "EV Network Website"})

In [6]:
unique_networks = full_and_clean["EV Network"].unique()
unique_networks

array(['Non-Networked', 'Volta', 'EV Connect', 'POWERFLEX',
       'ChargePoint Network', 'OpConnect', 'EVGATEWAY', 'CHARGELAB',
       'eVgo Network', 'AMPUP', 'EVCS', 'SWTCH', 'UNIVERSAL',
       'Blink Network', 'SHELL_RECHARGE', 'Tesla', 'ZEFNET',
       'Tesla Destination', 'Electrify America', 'FCN', 'LIVINGSTON',
       'FLO', 'FPLEV', '7CHARGE', 'RIVIAN_WAYPOINTS', 'BP_PULSE',
       'EVMATCH', 'RED_E', 'CIRCLE_K', 'AMPED_UP', 'EVRANGE',
       'GRAVITI_ENERGY', 'FLASH', 'RIVIAN_ADVENTURE', 'CHARGEUP', 'JULE',
       'NOODOE', 'CHARGIE', 'TURNONGREEN', 'LOOP', 'REVEL', 'POWER_NODE',
       'STAY_N_CHARGE', 'ENVIROSPARK', 'CHARGENET', 'EVBOLT', 'EVPOWER',
       'HONEY_BADGER'], dtype=object)

In [7]:
full_and_clean.to_csv("output/cleaned_stations.csv")

In [8]:
car_data = pd.read_csv("data/output.csv")

In [9]:
mean_range = car_data["Range (miles)"].mean()
mean_range

210.51456310679612

In [11]:
# Checking column names ignore
ev_clean_stations.head(1)

,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Station Phone,Status Code,Groups With Access Code,Access Days Time,...,Date Last Confirmed,ID,Updated At,Owner Type Code,Open Date,EV Connector Types,Country,Facility Type,EV Pricing,EV Workplace Charging
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,5:30am-9pm; pay lot,...,2023-01-10,1523,2023-02-14 15:54:11 UTC,P,1995-08-30,J1772,US,PARKING_GARAGE,Free; parking fee,False


In [17]:
# Snake casing columns to read in Database
snake_df = ev_clean_stations.rename(columns={'Fuel Type Code': 'Fuel_Type_Code',
                                             'Station Name': 'Station_Name',
                                               'Street Address': 'Street_Address',
                                                 'Station Phone': 'Station_Phone',
                                                   'Status Code': 'Status_Code',
                                                   'Groups With Access Code': 'Groups_With_Access_Code',
                                                   'Access Days Time': 'Access_Days_Time',
                                                   'Date Last Confirmed': 'Date_Last_Confirmed',
                                                   'Updated At': 'Updated_At',
                                                   'Owner Type Code': 'Owner_Type_Code',
                                                   'Open Date': 'Open_Date',
                                                   'EV Connector Types': 'EV_Connector_Types',
                                                   'Facility Type': 'Facility_Type',
                                                   'EV Pricing': 'EV_Pricing',
                                                   'EV Workplace Charging': 'EV_Worklplace_Charging'
                                                   })
print(f"Columns were renamed successfully.")

Columns were renamed successfully.


In [18]:
# Checking Dataframe to assure columns were renamed properly
snake_df.head(5)

,Fuel_Type_Code,Station_Name,Street_Address,City,State,ZIP,Station_Phone,Status_Code,Groups_With_Access_Code,Access_Days_Time,...,Date_Last_Confirmed,ID,Updated_At,Owner_Type_Code,Open_Date,EV_Connector_Types,Country,Facility_Type,EV_Pricing,EV_Worklplace_Charging
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,5:30am-9pm; pay lot,...,2023-01-10,1523,2023-02-14 15:54:11 UTC,P,1995-08-30,J1772,US,PARKING_GARAGE,Free; parking fee,False
1,ELEC,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,626-575-6800,E,Public,24 hours daily,...,2022-09-14,1583,2023-02-14 15:54:11 UTC,SG,1996-10-15,J1772,US,STATE_GOV,Free,False
2,ELEC,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,NaN,E,Public,24 hours daily,...,2023-01-10,6355,2023-02-14 15:54:11 UTC,P,1997-07-30,J1772,US,HOSPITAL,Free; parking fee,False
3,ELEC,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,800-256-6219,E,Public - Call ahead,Dealership business hours; customer use only,...,2023-08-10,6405,2024-01-31 22:07:01 UTC,P,2012-12-11,J1772,US,CAR_DEALER,NaN,False
4,ELEC,Galleria at Tyler,1299 Galleria at Tyler,Riverside,CA,92503,951-351-3110,E,Public,6am-12am daily,...,2023-09-14,6425,2024-01-31 22:07:01 UTC,P,1997-08-30,J1772,US,SHOPPING_MALL,Free,False


In [22]:
snake_df.to_csv('andy_snake_df', index=False)
print(f'Squidward Tentacles has the fanciest restaurant in Bikini Bottom, and he does not suck eggs. \n oh also the dataframe exported to CSV successfully')

Squidward Tentacles has the fanciest restaurant in Bikini Bottom, and he does not suck eggs. 
 oh also the dataframe exported to CSV successfully
